### ISC CamCan Suspense

This notebook implements the sliding-window ISC analysis on the CamCan data, and links the resulting dynamic ISC to reported suspense (obtained from an independent audience).

### setting up modules

In [1]:
import os, sys, scipy, nilearn, warnings
warnings.filterwarnings("ignore")
import numpy as np
from nilearn import plotting, input_data
from nilearn.input_data import NiftiLabelsMasker
import seaborn as sns
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline

from camcan_utils import *

### set up files

In [2]:
suspense_file         = '../data/avg_suspense.npy'
dict_file             = '../data/timeseries_dictionary.npy'
ts_data_file          = '../data/ts_data.npy'
shen_dictionary_file  = '../data/shen_dictionary.npy'
shen_atlas_filename   = '../data/shen_2mm_268_parcellation.nii'
sample_file           = '../data/func.nii'
dyn_isc_file          = '../data/dyn_isc_dict.npy'


### load data 
see description in notebook on main ISC analysis

In [3]:
ts_data = np.load(ts_data_file)
n_tr, n_regions, n_subjs = ts_data.shape
n_regions

268

### compute dynamic ISC analysis

In [ ]:
#''' 
### compute main sliding window analysis across full data (takes a while - hence precomputed)
dyn_isc_result_all = camcan_sliding_isc(ts_data[:,:,:])

#### set up two sub-samples (for consistency check)
subject_chunks = [[0, int(n_subjs/2)],
                  [int(n_subjs/2), n_subjs]]

n_iterations = len(subject_chunks)

dyn_isc_result = np.zeros((n_iterations, n_tr, n_regions))

sub_index = np.arange(n_subjs)
np.random.shuffle(sub_index)

for curr_sample in range(n_iterations): 
    print('Working on subgroup #', (curr_sample +1 ), ' of ', n_iterations)
    people_to_use = np.arange(subject_chunks[curr_sample][0], subject_chunks[curr_sample][1], 1)

    D = ts_data[:,:,sub_index[people_to_use] ]
    dyn_isc_result[curr_sample, :, :] = camcan_sliding_isc(D)

### save results - because computing them every time from scratch takes a lot of time 
### we save the output here so we can load it quicker later on

dyn_isc_dict = {}
dyn_isc_dict['dyn_isc_result_all'] = dyn_isc_result_all
dyn_isc_dict['dyn_isc_result_splithalf'] = dyn_isc_result

np.save('../data/dyn_isc_dict.npy', dyn_isc_dict) 

np.corrcoef( scipy.stats.zscore(np.nanmean(dyn_isc_result[0,:,:], axis = 1)), 
             scipy.stats.zscore(np.nanmean(dyn_isc_result[1,:,:], axis = 1)) )[0,1]

#'''

# if precomputed
#dyn_isc_dict       = np.load(dyn_isc_file).item() 
#dyn_isc_dict.keys()
#dyn_isc_result_all = dyn_isc_dict['dyn_isc_result_all']       #this is the result across the full sample
#dyn_isc_result     = dyn_isc_dict['dyn_isc_result_splithalf'] #this is the result for one random group split


Assuming 494 subjects with 193 time points and 268 voxel(s) or ROI(s).

1%

### plot dynISC: Whole brain (Figure 3A)

In [ ]:
f = plt.figure(figsize=(8,4))
ax1 = f.add_subplot(111)
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result_all[:,:], axis = 1)), color = 'black', label = 'dynISC across Brain (Whole Group)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[0,:,:],   axis = 1)), color = 'gray',  label = 'dynISC (Group1)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[1,:,:],   axis = 1)), color = 'gray',  label = 'dynISC (Group2)');
plt.legend(loc = 2)
plt.xlabel('Time')
plt.ylabel('low      dynamic ISC        high')
plt.ylim(-3, 4.5)
plt.xticks([]);
plt.yticks([]);
sns.despine()

### plot dynISC: aMCC (Figure 3B)

In [ ]:
curr_region = 220 #aMCC
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
print(curr_title)
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
ax1 = f.add_subplot(211)
ax1.plot(scipy.stats.zscore(dyn_isc_result_all[:,curr_region]), color = 'green',          label = 'dynISC aMCC (Whole Group)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[0,:,curr_region]),   color = 'mediumseagreen', label = 'dynISC aMCC (Group1)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[1,:,curr_region]),   color = 'mediumseagreen', label = 'dynISC aMCC (Group2)');
plt.legend(loc = 2)
plt.xlabel('Time')
plt.ylabel('low      dynamic ISC        high')
plt.ylim(-3, 4.5)
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'Greens',  annotate = False,cut_coords = curr_coords,
                                       axes = ax2);


### dynISC  vs CRM: Whole brain 

In [ ]:
f = plt.figure(figsize=(8,4))
ax1 = f.add_subplot(111)

suspense = np.load(suspense_file)

ax1.plot(scipy.stats.zscore(suspense),   color = 'red',   label = 'Suspense (CRM)' );
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result_all[:,:], axis = 1)), color = 'black', label = 'dynISC (all viewers)'   );
#ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[0,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group1)' );
#ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[1,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group2)');

plt.legend(loc = 0)
plt.ylim(-3, 4.3)
plt.xlabel('Time')
plt.ylabel('low       ISC | Suspsense (z-scored)        high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

alignment = np.corrcoef(
                (scipy.stats.zscore(np.nanmean(dyn_isc_result_all[:,:], axis = 1)) ),
                 scipy.stats.zscore(suspense) )[0,1]

print('Alignment: ' + str(np.round(alignment, 2)) )

### Regional dynISC  vs CRM: ACC

In [ ]:
curr_region = 220 
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
region_vector = np.zeros((1,268))
region_vector[0,curr_region] = 0.5
shen_masker = NiftiLabelsMasker(labels_img=shen_atlas_filename);
shen_masker.fit_transform(sample_file);
region_img = shen_masker.inverse_transform(region_vector);

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)

ax1.plot(scipy.stats.zscore(suspense), color = 'red',   label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(dyn_isc_result_all[:,curr_region]),             color = 'green', label = 'dynISC regional (all)');

plt.legend(loc = 0)
plt.ylim(-3, 3)
plt.xlabel('Time')
plt.ylabel('low   ISC | Suspsense (z-scored)    high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'Greens',  annotate = False,cut_coords = curr_coords,
                                       title = str(curr_region),axes = ax2);
    
print(np.corrcoef(
                    (scipy.stats.zscore(dyn_isc_result_all[:,curr_region]) ),
                     scipy.stats.zscore(suspense))[0,1]) 

### Regional dynISC  vs CRM: lateral frontal

In [ ]:
curr_region = 27
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
region_vector = np.zeros((1,268))
region_vector[0,curr_region] = 0.5
region_img = shen_masker.inverse_transform(region_vector);

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)
ax1.plot(scipy.stats.zscore(suspense), color = 'red',   label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(dyn_isc_result_all[:,curr_region]),             color = 'blue', label = 'dynISC regional (all)');

plt.legend(loc = 0)
plt.ylim(-3, 3)
plt.xlabel('Time')
plt.ylabel('low   ISC | Suspsense (z-scored)    high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'PuBu',  annotate = False,cut_coords = curr_coords,
                                       title = str(curr_region),axes = ax2);
    
print(np.corrcoef(
                    (scipy.stats.zscore(dyn_isc_result_all[:,curr_region]) ),
                     scipy.stats.zscore(suspense))[0,1]) 

### expand the regional ISC analysis to all regions

In [ ]:
isc_2_crm = np.zeros((1,n_regions))
for curr_region in range(n_regions):
    isc_2_crm[0,curr_region] = np.corrcoef(dyn_isc_result_all[:, curr_region], suspense)[0,1]

isc_2_crm_img = shen_masker.inverse_transform(isc_2_crm);
nilearn.plotting.plot_stat_map(nilearn.image.index_img(isc_2_crm_img,0), 
                               vmax = 0.8, 
                               draw_cross = True,
                               threshold = 0.5, #note this is only for illustration
                               cut_coords = (0,30,30),
                               #annotate = False,
                               title = "Relationship between regional dynISC and reported suspense"
                              );

view = nilearn.plotting.view_img(nilearn.image.index_img(isc_2_crm_img,0) ,
                                resampling_interpolation = 'linear',
                                threshold = 0.5);
view

In [ ]:
# load the dictionary and create the main data structure (ts_data) as well as auxilliary variables.
read_dictionary = np.load(dict_file).item()

subjs = list(read_dictionary.keys())
n_subjs = len(subjs)

n_tr, n_regions = read_dictionary[subjs[0]]['funcdata'].shape

print(n_subjs)
print(n_tr)
print(n_regions)

ts_data = np.zeros((n_tr, n_regions, n_subjs))
ts_data_undet = np.zeros((n_tr, n_regions, n_subjs))
motion_data = np.zeros((n_tr, 6, n_subjs))
age_data = np.zeros((n_subjs))
gender_data = np.zeros((n_subjs))

#loop over viewers and fill the ts_data array

for curr_sub in range(n_subjs):
    curr_sub_name = subjs[curr_sub]
    ts_data[:,:, curr_sub] =  read_dictionary[curr_sub_name]['funcdata']
    ts_data_undet[:,:, curr_sub] =  read_dictionary[curr_sub_name]['funcdata_unfilt_undet_stand']
    motion_data[:,:, curr_sub] =  read_dictionary[curr_sub_name]['motion']
    age_data[curr_sub] = read_dictionary[curr_sub_name]['age']
    gender_data[curr_sub] = read_dictionary[curr_sub_name]['gender']

### quantify movement

In [ ]:
n_mopar = 6
movement_quantification = np.zeros((n_mopar, n_subjs))

for subj in range(n_subjs):
    for mopar in range(n_mopar):
        
        movement_quantification[mopar, subj] = np.sum(np.abs(np.diff(motion_data[:,mopar,subj])))
        
plt.imshow(np.corrcoef(movement_quantification), cmap = 'seismic', vmin = -1, vmax = 1);
plt.colorbar();
plt.show()

# as can be seen, the by-subejct extent of motion is highly correlated across the six parameters
# if a subject has a lot of motion in one, it also has much in the other parameter (x,y,z, y, r, g)
# this provides the basis for collapsing them and ranking subjects in the overall amount.

#rank this
movement_quantification_ranks = np.zeros(movement_quantification.shape)

for parameter in range(movement_quantification.shape[0]):
    order = movement_quantification[parameter,:].argsort()
    movement_quantification_ranks[parameter,:] = order.argsort()

# derive an oerall metric
mean_rank = np.mean(movement_quantification_ranks, axis = 0)
order = mean_rank.argsort()
overall_rank = order.argsort()

# select the lowest ranking subjects (stifflers)
no_stiffler_indices = (np.where(overall_rank < 50))[0]
lo_stiffler_indices = np.setdiff1d(np.where( overall_rank < 100)[0], np.where(overall_rank < 50)[0] )

# select the highest ranking subjects (shakers)
high_shaker_indices = np.setdiff1d(np.where( overall_rank> (n_subjs-1) - 100)[0], np.where(overall_rank> (n_subjs-1) - 50)[0] )
highest_shaker_indices = (np.where(overall_rank > (n_subjs-1) - 50))[0]

# check that they differ markedly in movement
c = np.hstack((
     movement_quantification[:, no_stiffler_indices],
     movement_quantification[:, lo_stiffler_indices],
     movement_quantification[:, high_shaker_indices],
     movement_quantification[:, highest_shaker_indices]))

plt.imshow(scipy.stats.zscore(c.T).T);

In [ ]:
lo_stiffler_indices

In [ ]:
no_stiffler_indices

### compute dynISC for subgroups separately

In [ ]:
stiffler_shaker_list = [no_stiffler_indices,
                        lo_stiffler_indices,
                        high_shaker_indices,
                        highest_shaker_indices]
stiffler_shaker_list 

In [ ]:
n_iterations = len(stiffler_shaker_list) #stifflers vs. shakers
dyn_isc_result = np.zeros((n_iterations, n_tr, n_regions))

for curr_sample in range(n_iterations): 
    print('Working on subgroup #', (curr_sample +1 ), ' of ', n_iterations)
    people_to_use = stiffler_shaker_list[curr_sample]
    print('working with those indices ...')
    print(people_to_use)

    D = ts_data[:,:, people_to_use ]
    dyn_isc_result[curr_sample, :, :] = camcan_sliding_isc(D)

### save results - because computing them every time from scratch takes a lot of time 
### we save the output here so we can load it quicker later on

dyn_isc_dict = {}
dyn_isc_dict['dyn_isc_result_stiffler_shaker'] = dyn_isc_result


In [ ]:
f = plt.figure(figsize=(8,4))
ax1 = f.add_subplot(111)

ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[0,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group1)' );
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[1,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group2)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[2,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group2)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_result[3,:,:], axis = 1)),   color = 'gray',  label = 'dynISC (Group2)');


In [ ]:
curr_region = 6 
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
region_vector = np.zeros((1,268))
region_vector[0,curr_region] = 0.5
region_img = shen_masker.inverse_transform(region_vector);

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)
ax1.plot(scipy.stats.zscore(suspense), color = 'red',   label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[0,:,curr_region]),             color = 'blue', label = 'dynISC regional (lowest)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[1,:,curr_region]),             color = 'blue', label = 'dynISC regional (low)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[2,:,curr_region]),             color = 'blue', label = 'dynISC regional (high)');
ax1.plot(scipy.stats.zscore(dyn_isc_result[3,:,curr_region]),             color = 'red', label = 'dynISC regional (highest)');


plt.legend(loc = 0)
plt.ylim(-3, 3)
plt.xlabel('Time')
plt.ylabel('low   ISC | Suspsense (z-scored)    high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'PuBu',  annotate = False,cut_coords = curr_coords,
                                       title = str(curr_region),axes = ax2);
    
print(np.corrcoef(
                    (scipy.stats.zscore(dyn_isc_result[1,:,curr_region]) ),
                     scipy.stats.zscore(suspense))[0,1]) 

In [ ]:
for i in range(4):
    print(np.corrcoef(
                    (scipy.stats.zscore(dyn_isc_result[i,:,curr_region]) ),
                     scipy.stats.zscore(suspense))[0,1]) 

### ISC/dynISC on motion data

In [ ]:

D = motion_data
print('Calculating ISC on ', D.shape[1], ' regions/variables and ', D.shape[2], ' subjects')
motionISC = camcan_isc(D, summary_statistic=np.mean, verbose = True)
motionISC

In [ ]:
i = 2
m1mean = np.nanmean(motion_data[:,i,:248], axis = 1)
m2mean = np.nanmean(motion_data[:,i,248:], axis = 1)

f = plt.figure(figsize=(8,2))
plt.plot((m1mean), label = 'avg-group1');
plt.plot((m2mean), label = 'avg-group2');
plt.xlabel('time');
plt.ylabel('motion');
plt.legend(loc = 1)
plt.show()

print(np.corrcoef(m1mean, m2mean)[0,1])
# pretty surprising how reliable motion can be - at least in part.
# note that this is residual motion after pretty strict thresholding (see extraction notebook)
# however, it is considerably lower than the brain-ISC (see main ISC notebook where such group-averaged 
# timeseries are nearly 1 in all regions)

In [ ]:
subject_chunks = [[0, int(n_subjs/2)],
                  [int(n_subjs/2), n_subjs]]

n_iterations = len(subject_chunks)

D = motion_data;
dyn_motion_isc_result_all = camcan_sliding_isc(D)
dyn_motion_isc_result = np.zeros((n_iterations, n_tr, motion_data.shape[1]))

sub_index = np.arange(n_subjs)
np.random.shuffle(sub_index)

for curr_sample in range(n_iterations): 
    print('Working on subgroup #', (curr_sample +1 ), ' of ', n_iterations)
    people_to_use = np.arange(subject_chunks[curr_sample][0], subject_chunks[curr_sample][1], 1)

    D = motion_data[:,:,sub_index[people_to_use] ]
    dyn_motion_isc_result[curr_sample, :, :] = camcan_sliding_isc(D)


In [ ]:
f = plt.figure(figsize=(10,5))
ax1 = f.add_subplot(111)

#ax1.plot(scipy.stats.zscore(dyn_isc_result[0,14,:]), 
#         color = 'blue', label = 'Group1 d-ISC aMCC');
#ax1.plot(scipy.stats.zscore(dyn_isc_result[1,14,:]), 
#         color = 'blue', label = 'Group2 d-ISC aMCC');

ax1.plot(scipy.stats.zscore(np.mean(dyn_motion_isc_result[0,:,:], axis=1)), 
         alpha = 0.3, color = 'orange', label = 'Group1 d-ISC Motion');
ax1.plot(scipy.stats.zscore(np.mean(dyn_motion_isc_result[1,:,:], axis=1)), 
         alpha = 0.3, color = 'orange', label = 'Group2 d-ISC Motion');

ax1.plot(scipy.stats.zscore(suspense), color = 'red');
plt.legend();
plt.xlabel('Time');
plt.ylabel('ISC | Suspense');

In [ ]:
for i in range(6):
    print(np.corrcoef(scipy.stats.zscore(dyn_motion_isc_result[0,:,i]),
                              scipy.stats.zscore(suspense) )[0,1])

In [ ]:
print(np.corrcoef( scipy.stats.zscore(np.mean(dyn_motion_isc_result[0,:,:], axis=1)) ,
                              scipy.stats.zscore(suspense) )[0,1])



In [ ]:
np.corrcoef( scipy.stats.zscore(np.mean(dyn_motion_isc_result[0,:,:], axis=1))  ,
             scipy.stats.zscore( dyn_isc_result[0,:,6]  ) )[0,1]

### dynISC on lowMo/highMo subgroups

In [ ]:
n_iterations = len(stiffler_shaker_list) #stifflers vs. shakers
dyn_isc_motion_result = np.zeros((n_iterations, n_tr, motion_data.shape[1]))

for curr_sample in range(n_iterations): 
    print('Working on subgroup #', (curr_sample +1 ), ' of ', n_iterations)
    people_to_use = stiffler_shaker_list[curr_sample]
    print('working with those indices ...')
    print(people_to_use)

    #D = ts_data[:,:, people_to_use ]
    D = motion_data[:,:,people_to_use ]
    dyn_isc_motion_result[curr_sample, :, :] = camcan_sliding_isc(D)

### save results - because computing them every time from scratch takes a lot of time 
### we save the output here so we can load it quicker later on

dyn_isc_motion_dict = {}
dyn_isc_motion_dict['dyn_isc_result_stiffler_shaker'] = dyn_isc_motion_result


In [ ]:
curr_region = 6 
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
region_vector = np.zeros((1,268))
region_vector[0,curr_region] = 0.5
region_img = shen_masker.inverse_transform(region_vector);

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)
ax1.plot(scipy.stats.zscore(suspense), color = 'red',   label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(dyn_isc_motion_result[0,:,0]),             color = 'blue', label = 'dynISC regional (lowest)');
ax1.plot(scipy.stats.zscore(dyn_isc_motion_result[1,:,0]),             color = 'blue', label = 'dynISC regional (low)');
ax1.plot(scipy.stats.zscore(dyn_isc_motion_result[2,:,0]),             color = 'blue', label = 'dynISC regional (high)');
ax1.plot(scipy.stats.zscore(dyn_isc_motion_result[3,:,0]),             color = 'red', label = 'dynISC regional (highest)');


plt.legend(loc = 0)
plt.ylim(-3, 3)
plt.xlabel('Time')
plt.ylabel('low   ISC | Suspsense (z-scored)    high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'PuBu',  annotate = False,cut_coords = curr_coords,
                                       title = str(curr_region),axes = ax2);
    
for i in range(6):
    print(np.corrcoef(scipy.stats.zscore(dyn_isc_motion_result[0,:,i]),
                              scipy.stats.zscore(suspense) )[0,1])

In [ ]:
(dyn_isc_result[0,:,3])

In [ ]:
dyn_isc_result_all.shape

In [ ]:
dyn_motion_isc_result_all.shape

In [ ]:
np.corrcoef(np.nanmean(dyn_isc_result_all[:,:], axis=1), 
            np.nanmean(dyn_motion_isc_result_all[:,:], axis=1))

In [ ]:
from pandas import DataFrame
import statsmodels.api as sm

dd =           {'suspense': scipy.stats.zscore(suspense),
                'motion_overall': scipy.stats.zscore(np.nanmean(dyn_motion_isc_result_all[:,:], axis=1)) ,
                'dyn_isc_wholebrain': scipy.stats.zscore(np.nanmean(dyn_isc_result_all[:,:], axis=1)),
               }

df = DataFrame(dd ,columns=['suspense','motion_overall','dyn_isc_wholebrain']) 
sns.pairplot(df);

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

mod = smf.ols(formula = 'suspense ~ dyn_isc_wholebrain + motion_overall', data = df)
res = mod.fit()

res.summary()

In [ ]:
df.corr()

### reviewer comment auditory , visual

In [ ]:
curr_region = 196 
shen_dictionary = np.load(shen_dictionary_file).item()
curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 

f = plt.figure(figsize=(8,8))
region_vector = np.zeros((1,268))
region_vector[0,curr_region] = 0.5
region_img = shen_masker.inverse_transform(region_vector);

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)
ax1.plot(scipy.stats.zscore(suspense), color = 'red', linestyle = ':' , label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(dyn_isc_result_all[:,196]),             color = 'blue', label = 'dynISC auditory (all)');
ax1.plot(scipy.stats.zscore(dyn_isc_result_all[:,203]),             color = 'green', label = 'dynISC visual (all)');

plt.legend(loc = 0)
plt.ylim(-3, 3)
plt.xlabel('Time')
plt.ylabel('low   ISC | Suspsense (z-scored)    high')
plt.xticks([]);
plt.yticks([]);
sns.despine()

ax2 = f.add_subplot(212)    
nilearn.plotting.plot_stat_map(nilearn.image.index_img(region_img,0),threshold = 0.45, vmax = 1., draw_cross = True,colorbar = False,
                                       cmap = 'PuBu',  annotate = False,cut_coords = curr_coords,
                                       title = str(curr_region),axes = ax2);
    
print(np.corrcoef(
                    (scipy.stats.zscore(dyn_isc_result_all[:,curr_region]) ),
                     scipy.stats.zscore(suspense))[0,1]) 

In [ ]:
print(np.mean(dyn_isc_result_all[:,196]))
print(np.std(dyn_isc_result_all[:,196]))
print(np.min(dyn_isc_result_all[:,196]))
print(np.max(dyn_isc_result_all[:,196]))

### subgroup dynISC


In [ ]:
# create four groups
title_list = ['senior_females',
              'senior_males',
              'young_females',
              'young_males']

senior_females = np.intersect1d(np.where((gender_data == 2)), 
                  np.where(age_data > np.mean(age_data)), 
                  assume_unique=False)

senior_males = np.intersect1d(np.where((gender_data == 1)), 
                  np.where(age_data > np.mean(age_data)), 
                  assume_unique=False)

young_females = np.intersect1d(np.where((gender_data == 2)), 
                  np.where(age_data < np.mean(age_data)), 
                  assume_unique=False)

young_males = np.intersect1d(np.where((gender_data == 1)), 
                  np.where(age_data < np.mean(age_data)), 
                  assume_unique=False)

cond_list = [senior_females, senior_males, young_females, young_males]

# compute ISC and plot

n_iterations = len(cond_list) 
dyn_isc_subgroup_result = np.zeros((n_iterations, n_tr, n_regions))


for curr_sample in range(n_iterations):
    curr_subs = cond_list[curr_sample]
    print(curr_subs)
    D = ts_data[:,:,curr_subs]
    
    
    dyn_isc_subgroup_result[curr_sample, :, :] = camcan_sliding_isc(D)
    
    

In [ ]:
curr_region = 6

f = plt.figure(figsize=(8,8))

curr_title = shen_dictionary[curr_region + 1]['name'] 
curr_coords = shen_dictionary[curr_region + 1]['coords'] 
    
ax1 = f.add_subplot(211)
#ax1.plot(scipy.stats.zscore(suspense), color = 'red',   label = 'Suspense (CRM)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_subgroup_result[0,:,:], axis=1)),              label = 'dynISC regional (agesex1)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_subgroup_result[1,:,:], axis=1)),              label = 'dynISC regional (agesex1)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_subgroup_result[2,:,:], axis=1)),              label = 'dynISC regional (agesex1)');
ax1.plot(scipy.stats.zscore(np.nanmean(dyn_isc_subgroup_result[3,:,:], axis=1)),              label = 'dynISC regional (agesex1)');


### Permutation dynISC



In [ ]:
 
### compute main sliding window analysis across full data (takes a while - hence precomputed)

n_iterations = 100

#### set up two sub-samples (for consistency check)
subject_chunks = [[0, int(n_subjs/2)],
                  [int(n_subjs/2), n_subjs]]

n_subgroups = len(subject_chunks)

dyn_isc_result = np.zeros((n_iterations ,n_tr, n_regions))


for curr_iteration in range(n_iterations):
    print('Working on iteration #', (curr_iteration +1 ), ' of ', n_iterations)
    
    #randomize subject array
    sub_index = np.arange(n_subjs)
    np.random.shuffle(sub_index)

    people_to_use = np.arange(subject_chunks[0][0], subject_chunks[0][1], 1)
    D = ts_data[:,:,sub_index[people_to_use] ]
    dyn_isc_result[curr_iteration,0, :, :] = camcan_sliding_isc(D)
    
    

### save results - because computing them every time from scratch takes a lot of time 
### we save the output here so we can load it quicker later on

dyn_isc_dict = {}
dyn_isc_dict['dyn_isc_result_splithalf'] = dyn_isc_result

np.save('../data/dyn_isc_dict_resampled.npy', dyn_isc_dict) 


# if precomputed
#dyn_isc_dict = np.load(dyn_isc_file).item() 
#dyn_isc_dict.keys()
#dyn_isc_result_all = dyn_isc_dict['dyn_isc_result_all']
#dyn_isc_result = dyn_isc_dict['dyn_isc_result_splithalf']


In [ ]:
dyn_isc_result = dyn_isc_result[:50,:,:]

In [ ]:

dyn_isc_dict = {}
dyn_isc_dict['dyn_isc_result_splithalf'] = dyn_isc_result

np.save('../data/dyn_isc_dict_resampledLARGE.npy', dyn_isc_dict) 



In [ ]:
dyn_isc_result.shape

In [ ]:
dyn_isc_file = '../data/dyn_isc_dict_resampledF.npy'
dyn_isc_dict = np.load(dyn_isc_file).item() 
dyn_isc_result_f = dyn_isc_dict['dyn_isc_result_splithalf']


In [ ]:
dyn_isc_result_many = np.vstack((dyn_isc_result[:38,:,:], 
                                 #dyn_isc_result[48:,:,:],
           dyn_isc_result_a,
           dyn_isc_result_b,
            
           dyn_isc_result_c,
           dyn_isc_result_d,
           dyn_isc_result_e,
                                 dyn_isc_result_b[:2,:,:],
           dyn_isc_result_f               ))

In [ ]:
dyn_isc_result_many.shape

In [ ]:
dyn_isc_dict = {}
dyn_isc_dict['dyn_isc_result_splithalf'] = dyn_isc_result_many

np.save('../data/dyn_isc_dict_resampled100.npy', dyn_isc_dict) 


In [ ]:
plt.plot(np.squeeze(np.nanmean(dyn_isc_result_many, axis=2)).T);

In [ ]:
np.squeeze(np.nanmean(dyn_isc_result_many, axis=2)).shape

In [ ]:
d = np.hstack(np.squeeze(np.nanmean(dyn_isc_result_many, axis=2)))
h = np.hstack([np.arange(193)] * 100)
dh = np.vstack((d,h)).T
df_dh = pd.DataFrame(data =  dh, columns = ['a','b'])


f = plt.figure(figsize=(8,4));
sns.lineplot(x="b", 
             y="a", 
             color = 'red',
             ci = 'sd',
             data=df_dh);
plt.plot(np.nanmean(np.squeeze(np.nanmean(dyn_isc_result_many, axis=2)), axis=0), color ='red');
plt.ylabel('low                dynamic ISC                 high');
plt.xlabel('Time');
#plt.ylim(-2.5,2);
plt.xticks([]);
plt.yticks([]);
sns.despine();